In [61]:
import pandas as pd
import re
import unicodedata
import json
from fuzzywuzzy import process


In [62]:
csv_file_path = 'LISTAS VIGENTES\MONESIS GLOBAL CONTINGENCIAS AGOSTO 2024.csv'

In [63]:
try:
    df = pd.read_csv(csv_file_path, encoding='utf-8', delimiter=";")
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_path, encoding='latin1', delimiter=";")

df.dropna(how='all', inplace=True)

In [64]:
df.columns = df.columns.str.strip()
df.columns

Index(['ID', 'PRESTAMO', 'VEHICULO', 'FECHA DE ULT ACT', 'NOMBRE',
       'CONTINGENCIA', 'IRRECUPERABLES', 'DICTAMEN DE IRRECUPERABILIDAD',
       'CONSOLIDADO', 'MACRO ETAPA ACTUALIZADA', 'MICRO ETAPA ACTUALIZADA',
       'EXPEDIENTE', 'JUZGADO', 'PLAZA JURISDICCION', 'ESTADO JURISDICCION',
       'DESPACHO', 'SALDO INFONAVIT ACTUALIZADO', 'COFI',
       'AVALUO ORIGINAL_ORG', 'AVALUO REFERENCIA_ACT', 'MONEDA', 'GTIA_EDO',
       'GTIA_MPIO', 'GTIA_COLONIA', 'GTIA_UBICACION', 'GTIA_CODPOSTAL',
       'M2 TERRENO', 'M2 CONSTRUCCION', 'FECHA ORIGEN',
       'IMPORTE CONCEDIDO VALORIZADO', 'CAPITAL ACT', 'INT ORD ACT',
       'SDO CONT ACT', 'INT MOR + ACC ACT', 'ADEUD TOT ACT',
       'Precio individual nuevo', 'Precio Mejorado paquete nuevo'],
      dtype='object')

In [65]:
def rename_columns(df):

    df_renamed = df.rename(columns={
        'ID': 'FOLIO',
        'MICRO ETAPA ACTUALIZADA': 'ETAPA',
        'GTIA_EDO': 'ESTADO',
        'GTIA_MPIO': 'MUNICIPIO',
        'GTIA_COLONIA': 'COLONIA',
        'GTIA_UBICACION': 'CALLE_Y_NUMERO',
        'GTIA_CODPOSTAL': 'CP',
        'M2 CONSTRUCCION': 'CONSTRUCCION',
        'M2 TERRENO': 'TERRENO',
        'Precio individual nuevo': '2DO_PAGO'
    })
    return df_renamed

def add_columns(df):
    df["LISTA"] = "MONESIS GLOBAL"

    return df


df = rename_columns(df)
df = df.apply(add_columns, axis=1)

<h3>LIMPIEZA DE NUMEROS</h3>

In [66]:
def number_cleaning(df):
    # Limpiar caracteres no numéricos en 'FOLIO' y 'CP'
    df["FOLIO"] = df["FOLIO"].replace({r'\D': ''}, regex=True)
    df["CP"] = df["CP"].replace({r'\D': ''}, regex=True)

    # Convertir a números, manejando errores y valores no convertibles
    df["FOLIO"] = pd.to_numeric(df["FOLIO"], errors='coerce')
    df["CP"] = pd.to_numeric(df["CP"], errors='coerce')

    # Rellenar valores NaN con 0 y convertir a enteros
    df["FOLIO"] = df["FOLIO"].fillna(0).astype(int)
    df["CP"] = df["CP"].fillna(0).astype(int)

    # Limpiar caracteres en '2DO_PAGO' y convertir a float
    df["2DO_PAGO"] = df["2DO_PAGO"].replace({r'\$': '', ',': ''}, regex=True)
    df["2DO_PAGO"] = pd.to_numeric(df["2DO_PAGO"], errors='coerce')  # Convertir a float

    return df

df = number_cleaning(df)

<h3>LIMPIEZA DE TEXTO</h3>

In [67]:
from word2number_es import w2n

def remove_punctuation(text):
    if isinstance(text, str):
        clean = re.sub(r'[\-:;]', ' ', text)
        clean_spaces = ' '.join(clean.split())
        return clean_spaces
    
    return text

"""def safe_word_to_number(text):
    if isinstance(text, str):
        try:
            return word_to_number(text)
        except RecursionError:
            return text
    return text"""

df["COLONIA"] = df["COLONIA"].apply(remove_punctuation)
df["CALLE_Y_NUMERO"] = df["CALLE_Y_NUMERO"].apply(remove_punctuation)
df["MUNICIPIO"] = df["MUNICIPIO"].apply(remove_punctuation)
df["ESTADO"] = df["ESTADO"].apply(remove_punctuation)

<h3>NORMALIZAR DIRECCIONES</h3>

In [68]:
def normalizar_direccion(df):
    direccion = df["CALLE_Y_NUMERO"]
    # Asegurarse de que la dirección no sea NaN
    if pd.isna(direccion):
        return ''
    
    # Eliminar las comas
    direccion = direccion.replace(',', '')
    
    # Añadir espacio antes y después de números cuando no haya espacio
    direccion = re.sub(r'(?<=\D)(?=\d)', ' ', direccion)
    direccion = re.sub(r'(?<=\d)(?=\D)', ' ', direccion)
    
    # Añadir punto a "ESQ" solo si no lo tiene
    direccion = re.sub(r'\bESQ\b(?!\.)', 'ESQ.', direccion, flags=re.IGNORECASE)
    
    # Reemplazar comillas dobles con comillas simples o eliminarlas
    direccion = direccion.replace('""', '"')
    direccion = direccion.replace('"', '')
    
    # Normalizar caracteres Unicode
    direccion = unicodedata.normalize('NFKC', direccion)
    
    # Reemplazar abreviaturas específicas, considerando todas las combinaciones de mayúsculas y minúsculas
    direccion = re.sub(r'\bNUM\. EXT\.\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUM. EXT."
    direccion = re.sub(r'\s*\.\s*', ' ', direccion)
    direccion = re.sub(r'\bEXT\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUM. EXT."
    direccion = re.sub(r'\bEXT\. \b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUM. EXT."
    direccion = re.sub(r'\bNUM\.\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUM."
    direccion = re.sub(r'\bEXT\.\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUM."
    direccion = re.sub(r'\bNUM\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUM."
    direccion = re.sub(r'\bNUMERO\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUMERO"
    direccion = re.sub(r'\bOf\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUMERO"
    direccion = re.sub(r'\bNo\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUMERO"
    direccion = re.sub(r'\bNUMERO\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUMERO"
    direccion = re.sub(r'\bNUM\. INT\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "NUM. INT"
    direccion = re.sub(r'\bLt\b', 'Lote', direccion, flags=re.IGNORECASE)  # Reemplazar "Lt." por "Lote"
    direccion = re.sub(r'\bL\.T\.\b', 'Lote', direccion, flags=re.IGNORECASE)  # Reemplazar "L.T." por "Lote"
    direccion = re.sub(r'\bLote\b', 'Lote', direccion, flags=re.IGNORECASE)  # Reemplazar "Lote" sin abreviatura
    direccion = re.sub(r'\bMZA\b', 'Manzana', direccion, flags=re.IGNORECASE)  # Reemplazar "Mza." por "Manzana"
    direccion = re.sub(r'\bM\.Z\.A\.\b', 'Manzana', direccion, flags=re.IGNORECASE)  # Reemplazar "M.Z.A." por "Manzana"
    direccion = re.sub(r'\bManzana\b', 'Manzana', direccion, flags=re.IGNORECASE)  # Reemplazar "Manzana" sin abreviatura
    
    # Reemplazar abreviaturas de calles
    direccion = re.sub(r'\bCalle\b', '', direccion, flags=re.IGNORECASE)  # Eliminar "Calle"
    direccion = re.sub(r'\bAv\.?\b', 'Avenida', direccion, flags=re.IGNORECASE)  # Reemplazar "Av." por "Avenida"
    
    # Reemplazar cualquier " - " o " . " por un espacio simple
    direccion = re.sub(r'\s*-\s*', ' ', direccion)
    direccion = re.sub(r'\s*\.\s*', ' ', direccion)
    # Asegurar que los espacios múltiples sean reducidos a uno solo
    direccion = re.sub(r'\s+', ' ', direccion).strip()
    
    return df

In [69]:
# Diccionario de abreviaturas
abbreviations = {
    'C.P': 'Código Postal',
    'CP': 'Código Postal',

    'MZ': 'Manzana',
    'MZA': 'Manzana',
    'M': 'Manzana',

    'SM': 'Supermanzana',
    'S.M': 'Supermanzana',

    'U. H.': 'Unidad habitacional',

    'LTE': 'Lote',
    'L': 'Lote',

    'C': 'Calle',
    'CALL': 'Calle',

    'NUM': 'Número',
    'NÚM': 'Número',
    "NO": "Número",


    'AV': 'Avenida',
    'AVE': 'Avenida',

    'BLVD': 'Boulevard',

    'PL': 'Plaza',

    'PARQ': 'Parque',

    'EDIF': 'Edificio',

    'DEPTO': 'Departamento',

    'UNID': 'Unidad',

    'CIRC': "Circuito",

    "HDA": "Hacienda",

    "ED": "Edificio",

    "DEP": "Departamento",
    "DPTO": "Departamento",

    "FRAC": "Fraccionamiento",
    "FRACC": "Fraccionamiento",

    "EXT": "Exterior",

    "INT": "Interior",

    "VIV": "Vivienda",

    "COL": "Colonia"

}

# Función para limpiar direcciones
def clean_street(df):
    street = df["CALLE_Y_NUMERO"]
    street = str(street).upper()
    
    for abbreviation, full_form in abbreviations.items():
        # Usar expresión regular para reemplazar abreviaturas con o sin punto al final
        pattern = r'\b' + re.escape(abbreviation) + r'\b\.?'
        street = re.sub(pattern, full_form, street, flags=re.IGNORECASE)
    
    # Opcional: Eliminar dobles espacios resultantes de reemplazos
    street = ' '.join(street.split())
    df["CALLE_Y_NUMERO"] = street
    
    return df

In [58]:
df_inegi = pd.read_csv("INEGI.csv", delimiter=",")
df_inegi
def get_location(df):
    # Extract information from the row
    cp = df['CP']  # Postal Code
    #street = row['Dirección']  # Street name
    district = df['COLONIA']  # District
    municipality = df['MUNICIPIO']  # Municipality
    state = df['ESTADO']  # State

    if pd.notna(cp):
        cp = str(cp).zfill(5)
    
    df['CP'] = int(cp)
    
    # Filter the dataset to get data related to the Postal Code
    df_info_cp = df_inegi[df_inegi["cp"] == int(cp)]
    if df_info_cp.empty:
        df["CP"] = "Revisar CP " + cp
    else:        
        # Get lists for comparison
        list_district = df_info_cp["colonia"].tolist()
        list_municipality = df_info_cp["municipio"].tolist()
        list_state = df_info_cp["estado"].tolist()


        if pd.notna(district) and pd.notna(municipality) and pd.notna(state):
            municipality_match, percentage = process.extractOne(municipality, list_municipality)
            if percentage < 50:  # Conditional to match the CP with the municipality based on text similarity
                district_match = process.extractOne(district, list_district)
                state_match = process.extractOne(state, list_state)
                df['COLONIA'] = "REVISAR "+district_match[0]
                df['MUNICIPIO'] = "REVISAR "+municipality_match
                df['ESTADO'] = "REVISAR "+state_match[0]
            else:
                # Compare and normalize district, municipality, and state
                district_match = process.extractOne(district, list_district)
                state_match = process.extractOne(state, list_state)

                # Append the normalized data
                df['COLONIA'] = district_match[0] if district_match else None
                df['MUNICIPIO'] = municipality_match if municipality_match else None
                df['ESTADO'] = state_match[0] if state_match else None
        else:
            print("Values not found")
        # Convert all elements to uppercase
        #output_list = [str(item).upper() if isinstance(item, str) else item for item in normalized_data]
    return df

In [70]:
df = df.apply(get_location, axis = 1)
df = df.apply(clean_street, axis = 1)

df = df.apply(normalizar_direccion, axis=1)

Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found
Values not found


In [71]:
def clasificar_etapa(df):
    adjudicada = [
        "Auto Aprobatorio",
        "Validacion De Documentacion",
        "Escritura Vehiculo No Inscrita",
        "Adjudicado Originador",
        "Escritura Originador No Inscrita",
        "Termino De Proceso",
        "Dacion En Pago",
        "Adjudicación A Favor Del Originador",
        "Escritura A Favor Del Vehículo Inscrita",
        "Adjudicación A Favor Del Vehículo",
        "Escritura A Favor Del Vehículo No Inscrita",
        "Escritura A Favor Del Originador No Inscrita",
        "Adjudicación",
        "Escritura A Favor Del Originador Inscrita",
        "Adjudicado Vehiculo",
        "Adjudicado Originador",
        "Escritura Vehiculo Inscrita",
        "Escritura Originador Inscrita",
        "Toma De Posesion"
    ]

    sentencia = [
        "Valorizacion",
        "Fecha De Remate",
        "Audiencia De Remate",
        "Sentencia Firme",
        "Sentencia Primera Instancia",
        "Publicacion De Edictos",
        "Sentencia"
    ]

    juicio = [
        "Emplazamiento Por Edictos",
        "Notificacion De Cesion",
        "Diligenciacion De Exhorto",
        "Presentacion De Demanda",
        "Auto Admisorio",
        "Desahogo De Pruebas",
        "Contestacion",
        "Oficios Dependencias",
        "Apelacion Sentencia Primera Instancia",
        "Juicio De Amparo Sentencia",
        "Apelacion Auto Admisorio",
        "Citado Para Sentencia"
    ]

    cobranza = [
        "Busqueda De Documentos",
        "Certificacion Contable",
        "Sin Dato"
    ]
    

    etapa = str(df["ETAPA"]).strip()  # Convert to string and strip whitespace
    
    # Match against each list using fuzzy matching
    match_adjudicada, score_adjudicada = process.extractOne(etapa, adjudicada)
    match_sentencia, score_sentencia = process.extractOne(etapa, sentencia)
    match_juicio, score_juicio = process.extractOne(etapa, juicio)
    match_cobranza, score_cobranza = process.extractOne(etapa, cobranza)

    # Determine the classification based on the highest score
    if score_adjudicada >= 80:
        df["ETAPA_ALTALTIUM"] = "ADJUDICADA"
    elif score_sentencia >= 80:
        df["ETAPA_ALTALTIUM"] = "SENTENCIA"
    elif score_juicio >= 80:
        df["ETAPA_ALTALTIUM"] = "JUICIO"
    elif score_cobranza >= 80:
        df["ETAPA_ALTALTIUM"] = "COBRANZA"
    else:
        df["ETAPA_ALTALTIUM"] = "DESCONOCIDA"

    return df

df = df.apply(clasificar_etapa, axis = 1)

In [72]:
# Definir las columnas del inventario
columns = [
    "LISTA", "FOTOS", "FICHAS", "FOLIO", "TIPO", "CALLE_Y_NUMERO", "COLONIA",
    "ALCALDIA/MUNICIPIO", "ESTADO", "CP", "TERRENO", "CONSTRUCCIÓN", 
    "RECAMARAS", "SANITARIOS", "ESTACIONAMIENTOS", "1ER_PAGO", 
    "2DO_PAGO", "TOTAL", "VALOR_APROX", "ALREDEDORES", "GOOGLE_MAPS", 
    "CALIF", "CALIF_EV", "LATITUD", "LONGITUD", "ETAPA", "ESTADO_DE_VIVIENDA",
]

In [73]:
# Comprobar las columnas
columns_to_check = ['RECAMARAS', 'SANITARIOS', 'ESTACIONAMIENTOS']

for column in columns_to_check:
    if column not in df.columns:
        df[column] = pd.NA  # O puedes usar `None` si prefieres

In [74]:
inventory_df = pd.DataFrame(columns=columns, index=df.index)

inventory_df["LISTA"] = df["LISTA"]
inventory_df["ETAPA"] = df["ETAPA_ALTALTIUM"]

inventory_df["FOLIO"] = df["FOLIO"].apply(lambda x: abs(x)) #Usé abs porque en la limpieza de numero se le agrega -
# inventory_df["TIPO"] = df["Tipo Inmueble"].str.strip().str.title() No hay tipo en global
inventory_df["ESTADO"] = df["ESTADO"].str.strip().str.title()
inventory_df["ALCALDIA/MUNICIPIO"] = df["MUNICIPIO"].str.strip().str.title()
inventory_df["COLONIA"] = df["COLONIA"].str.strip().str.title()
inventory_df["CALLE_Y_NUMERO"] = df["CALLE_Y_NUMERO"].str.strip().str.title()
inventory_df["CP"] = df["CP"]
inventory_df["TERRENO"] = df["TERRENO"]
inventory_df["CONSTRUCCIÓN"] = df["CONSTRUCCION"]
inventory_df["2DO_PAGO"] = df["2DO_PAGO"]

In [75]:
def calcular_primer_pago(salida_al_mercado):
    if salida_al_mercado <= 499000:
        return 226000
    elif 500000 <= salida_al_mercado <= 1499000:
        return 395000
    elif salida_al_mercado >= 1500000:
        return 474000
    else:
        return None

inventory_df["1ER_PAGO"] = df["2DO_PAGO"].apply(calcular_primer_pago)

inventory_df["1ER_PAGO"].astype(int)

0      226000
1      226000
2      226000
3      226000
4      226000
        ...  
314    226000
315    226000
316    226000
317    226000
318    226000
Name: 1ER_PAGO, Length: 319, dtype: int32

In [76]:
inventory_df["TOTAL"] = inventory_df["1ER_PAGO"] + inventory_df["2DO_PAGO"]

inventory_df["2DO_PAGO"] = inventory_df["2DO_PAGO"].apply(lambda x: f'{x:.0f}' if x.is_integer() else f'{x:.2f}')
inventory_df["TOTAL"] = inventory_df["TOTAL"].apply(lambda x: f'{x:.0f}' if x.is_integer() else f'{x:.2f}')

In [77]:
def calculate_rooms(construccion, recamaras):
    # Intenta convertir construccion a número
    try:
        construccion = float(construccion)
    except ValueError:
        construccion = pd.NA  # Si la conversión falla, asignar NA
    
    # Si recamaras tiene un valor válido, devuelve ese valor
    if pd.notna(recamaras) and recamaras != 0:
        return recamaras
    elif pd.isna(recamaras) or recamaras == 0:
        if pd.notna(construccion):  # Solo hacer comparaciones si construccion es numérico
            if construccion <= 39:
                return 1
            elif 40 <= construccion <= 80:
                return 2
            elif 81 <= construccion <= 249:
                return 3
            elif 250 <= construccion <= 349:
                return 4
            elif construccion >= 350:
                return 5
    return pd.NA  # Optional: return NA if construccion is not valid


inventory_df["RECAMARAS"] = inventory_df.apply(lambda row: calculate_rooms(row["CONSTRUCCIÓN"], row["RECAMARAS"]), axis=1)

In [78]:
def calculate_bathrooms(construccion, sanitarios):
    # Intenta convertir construccion a número
    try:
        construccion = float(construccion)
    except ValueError:
        construccion = pd.NA  # Si la conversión falla, asignar NA
    
    # Si sanitarios tiene un valor válido, devuelve ese valor
    if pd.notna(sanitarios) and sanitarios != 0:
        return sanitarios
    elif pd.isna(sanitarios) or sanitarios == 0:
        if pd.notna(construccion):  # Solo hacer comparaciones si construccion es numérico
            if construccion <= 69:
                return 1
            elif 70 <= construccion <= 99:
                return 1.5
            elif 100 <= construccion <= 159:
                return 2
            elif 160 <= construccion <= 209:
                return 2.5
            elif 210 <= construccion <= 250:
                return 3
            elif 251 <= construccion < 300:
                return 3.5
            elif construccion >= 300:
                return 4
    return pd.NA  # Optional: return NA if construccion is not valid

# Supongamos que `inventory_df` es tu DataFrame
inventory_df["SANITARIOS"] = inventory_df.apply(lambda row: calculate_bathrooms(row["CONSTRUCCIÓN"], row["SANITARIOS"]), axis=1)


In [79]:
def calculate_parking(construccion, estacionamientos):
    # Intenta convertir construccion a número
    try:
        construccion = float(construccion)
    except ValueError:
        construccion = pd.NA  # Si la conversión falla, asignar NA
    
    # Si estacionamientos tiene un valor válido, devuelve ese valor
    if pd.notna(estacionamientos) and estacionamientos != 0:
        return estacionamientos
    elif pd.isna(estacionamientos) or estacionamientos == 0:
        if pd.notna(construccion):  # Solo hacer comparaciones si construccion es numérico
            if construccion <= 99:
                return 1
            elif 100 <= construccion <= 249:
                return 2
            elif 250 <= construccion <= 359:
                return 3
            elif construccion >= 360:
                return 4
    return pd.NA  # Optional: return NA if construccion is not valid

# Supongamos que `inventory_df` es tu DataFrame
inventory_df["ESTACIONAMIENTOS"] = inventory_df.apply(lambda row: calculate_parking(row["CONSTRUCCIÓN"], row["ESTACIONAMIENTOS"]), axis=1)


In [80]:
#On ths versión, we need to add the state, colony, and cp to the google maps search

def generar_enlace_busqueda_google_maps(row):
    direccion = f"{row['CALLE_Y_NUMERO']}, {row['COLONIA']}, {row['ALCALDIA/MUNICIPIO']}, {row['ESTADO']}, {row['CP']}"
    direccion = re.sub(r'[\""]', '', direccion)
    direccion = re.sub(r'\s+', ' ', direccion).strip()  # Normalizar espacios múltiples
    return f"https://www.google.com/maps/search/?api=1&query={direccion.replace(' ', '+')}"

# Aplicar la función a cada fila del DataFrame
inventory_df["GOOGLE_MAPS"] = inventory_df.apply(generar_enlace_busqueda_google_maps, axis=1)

In [81]:
inventory_json = inventory_df.to_json(orient="records", force_ascii=False)

print(json.dumps(json.loads(inventory_json), indent=4, ensure_ascii=False))

[
    {
        "LISTA": "MONESIS GLOBAL",
        "FOTOS": null,
        "FICHAS": null,
        "FOLIO": 1582513741,
        "TIPO": null,
        "CALLE_Y_NUMERO": "Código Postal 77524 Departamento02 Edificio 06 \"17, 18 Y 19 \" 21 Benito Juarez Benito Juarez Código Postal 77524 Quintana Roo",
        "COLONIA": null,
        "ALCALDIA/MUNICIPIO": "Benito Juarez",
        "ESTADO": "Quintana Roo",
        "CP": 77524,
        "TERRENO": 1075.4,
        "CONSTRUCCIÓN": "66.48",
        "RECAMARAS": 2,
        "SANITARIOS": 1,
        "ESTACIONAMIENTOS": 1,
        "1ER_PAGO": 226000,
        "2DO_PAGO": "37200",
        "TOTAL": "263200",
        "VALOR_APROX": null,
        "ALREDEDORES": null,
        "GOOGLE_MAPS": "https://www.google.com/maps/search/?api=1&query=Código+Postal+77524+Departamento02+Edificio+06+17,+18+Y+19+21+Benito+Juarez+Benito+Juarez+Código+Postal+77524+Quintana+Roo,+nan,+Benito+Juarez,+Quintana+Roo,+77524",
        "CALIF": null,
        "CALIF_EV": null,
      